In [332]:
import os
import pdfplumber
import pandas as pd
# os.chdir("..")
os.getcwd()

'd:\\Project\\invoice_pdf2csv'

In [333]:
amazon_bboxes = {
    
    "bill_to_name": (313, 190, 558, 250)
    
}

In [334]:
pdf_path = r"input\amazon1.pdf"
invoice = {}

In [335]:
def extract_text_from_bbox(pdf_path, bounding_box):
    """
    Extracts text from a specified bounding box within a PDF page.
    """
    extracted_text = None
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]  # Assuming data is on the first page

        # Use the extract_text() method with the bbox parameter
        cropped_page = page.crop(bounding_box)
        extracted_text = cropped_page.extract_text()
            
    return extracted_text

In [336]:
import camelot

stream_tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream', suppress_stdout=False)
list_of_dfs = []
for i, table in enumerate(stream_tables):
    df = table.df.copy()
    list_of_dfs.append(df)

df = list_of_dfs[0]
seller_details = " ".join(df[0])
buyer_details = " ".join(df[1])


CropBox missing from /Page, defaulting to MediaBox


In [337]:
invoice["seller name"], invoice["seller_address"] = [item.strip() for item in seller_details.split("Sold By :", 1)[1].split("PAN No:", 1)[0].strip().split("*")]

In [338]:
invoice["pan_number"] = seller_details.split("PAN No:", 1)[1].split("GST Registration No:", 1)[0].strip()

In [339]:
invoice["gst_number"] = seller_details.split("GST Registration No:", 1)[1].split("Order Number:", 1)[0][:15].strip()

In [340]:
invoice["order_id"] = seller_details.split("Order Number:", 1)[1].split("Order Date:", 1)[0].strip()

In [341]:
invoice["order_date"] = seller_details.split("Order Date:", 1)[1].strip()

In [342]:
buyer_details

'Billing Address : Ashish Srivastava  Block A Flat 201 Sonestaa Meadows thubrahalli, Whitefield bangalore  Bengaluru, KARNATAKA, 560066  IN  State/UT Code: 29    Shipping Address : Ashish Srivastava Ashish Srivastava Block A Flat 201 Sonestaa Meadows thubrahalli,  Whitefield bangalore  Bengaluru, KARNATAKA, 560066 IN State/UT Code: 29 Place of supply: KARNATAKA Place of delivery: KARNATAKA Invoice Number : LKO1-2209551 Invoice Details : UP-LKO1-1931441115-2324 Invoice Date : 04.12.2023'

In [343]:
invoice["bill_to_name"] = df[1][1].strip()

In [344]:
invoice["bill_to_address"] = buyer_details.split(invoice['bill_to_name'], 1)[1].split("State/UT Code:", 1)[0].strip()

In [345]:
ship_to_name = extract_text_from_bbox(pdf_path, amazon_bboxes["bill_to_name"]).split("Shipping Address :", 1)[1].split("\n")[1:]

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [346]:
invoice["ship_to_address"] = [buyer_details.split(string)[1] for string in ship_to_name][1].split("State/UT Code:", 1)[0].strip()

In [347]:
invoice["ship_to_name"] = ship_to_name[1]

In [348]:
invoice

{'seller name': 'RETAILEZ PRIVATE LIMITED',
 'seller_address': 'Khasra numbers:444(P),445(P),459(P),  460,461,462,463,464,  465,466,467,468,469,470,471,472,473,474,,,  75(P),476,477,478, 479,480,  481,482,483(P),491,492,493(P) Village - Bhaukapur, Lucknow, Uttar Pradesh, 226401 IN',
 'pan_number': 'AALCR3173P',
 'gst_number': '09AALCR3173P1Z',
 'order_id': '405-8002819-1037154',
 'order_date': '04.12.2023',
 'bill_to_name': 'Ashish Srivastava',
 'bill_to_address': 'Block A Flat 201 Sonestaa Meadows thubrahalli, Whitefield bangalore  Bengaluru, KARNATAKA, 560066  IN',
 'ship_to_address': 'Block A Flat 201 Sonestaa Meadows thubrahalli, Whitefield bangalore  Bengaluru, KARNATAKA, 560066  IN',
 'ship_to_name': 'Ashish Srivastava'}

In [349]:
df

,0,1
0,Sold By :,Billing Address :
1,RETAILEZ PRIVATE LIMITED,Ashish Srivastava
2,,
3,"* Khasra numbers:444(P),445(P),459(P),","Block A Flat 201 Sonestaa Meadows thubrahalli,"
4,,Whitefield bangalore
5,"460,461,462,463,464,",
6,,"Bengaluru, KARNATAKA, 560066"
7,"465,466,467,468,469,470,471,472,473,474,,,",
8,,IN
9,"75(P),476,477,478, 479,480,",


In [350]:
invoice["invoice_number"] = buyer_details.split("Invoice Details :", 1)[1].split("Invoice Date :", 1)[0].strip()

In [351]:
invoice["invoice_date"] = buyer_details.split("Invoice Date :", 1)[1].strip()

In [352]:
all_extracted_tables = []
with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]
        tables = page.extract_tables()
if tables:
        for i, table_data in enumerate(tables):
                df = pd.DataFrame(table_data[1:], columns=table_data[0])
                all_extracted_tables.append(df)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [353]:
invoice["product_name"] = " ".join(all_extracted_tables[0]["Description"][0].split("HSN:", 1)[0].split("\n")).strip()

In [354]:
invoice["product_category"] = None

In [355]:
all_extracted_tables[0]

,Sl.\nNo,Description,Unit\nPrice,Qty,Net\nAmount,Tax\nRate,Tax\nType,Tax\nAmount,Total\nAmount
0,1,Gear Campus8 25L Medium Water Resistant School...,₹550.00,1,₹550.00,18%,IGST,₹99.00,₹649.00
1,TOTAL:,None,None,None,None,None,None,₹99.00,₹649.00
2,Amount in Words:\nSix Hundred Forty-nine only,None,None,None,None,None,None,None,None
3,For RETAILEZ PRIVATE LIMITED:\nAuthorized Sign...,None,None,None,None,None,None,None,None


In [356]:
invoice["product_taxable_value"] = all_extracted_tables[0]["Net\nAmount"][0].strip("₹")

In [357]:
invoice["product_taxable_value"]

'550.00'

In [358]:
invoice["GST"] = str(pd.to_numeric(all_extracted_tables[0]["Tax\nAmount"].astype(str).str.strip("₹"), errors='coerce').sum())

In [359]:
invoice["product_quantity"] = all_extracted_tables[0]["Qty"][0]

In [360]:
invoice["product_total"] = all_extracted_tables[0]["Total\nAmount"][0].strip("₹")

In [361]:
invoice

{'seller name': 'RETAILEZ PRIVATE LIMITED',
 'seller_address': 'Khasra numbers:444(P),445(P),459(P),  460,461,462,463,464,  465,466,467,468,469,470,471,472,473,474,,,  75(P),476,477,478, 479,480,  481,482,483(P),491,492,493(P) Village - Bhaukapur, Lucknow, Uttar Pradesh, 226401 IN',
 'pan_number': 'AALCR3173P',
 'gst_number': '09AALCR3173P1Z',
 'order_id': '405-8002819-1037154',
 'order_date': '04.12.2023',
 'bill_to_name': 'Ashish Srivastava',
 'bill_to_address': 'Block A Flat 201 Sonestaa Meadows thubrahalli, Whitefield bangalore  Bengaluru, KARNATAKA, 560066  IN',
 'ship_to_address': 'Block A Flat 201 Sonestaa Meadows thubrahalli, Whitefield bangalore  Bengaluru, KARNATAKA, 560066  IN',
 'ship_to_name': 'Ashish Srivastava',
 'invoice_number': 'UP-LKO1-1931441115-2324',
 'invoice_date': '04.12.2023',
 'product_name': 'Gear Campus8 25L Medium Water Resistant School Bag/Casual Backpack/Daypack/College Bag/for Men/Women - Green Grey | B019FBEJCO ( B019FBEJCO )',
 'product_category': Non